<a href="https://colab.research.google.com/github/mattg274/DS2002/blob/main/StockAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:

import json
import pandas as pd
import json
import requests

In [39]:
stock=input("Enter Ticker Symbol")

Enter Ticker SymbolAAPL


In [40]:
print("The Stock we will research is:" + stock)

The Stock we will research is:AAPL


In [41]:
apikey = "X"

url_1 = "https://yfapi.net/v6/finance/quote"
querystring = {"symbols": stock}
headers = {'x-api-key': apikey}

response_1 = requests.get(url_1, headers=headers, params=querystring)

stock_data = {
    "Symbol": stock,
    "Company Name": None,
    "Current Price": None,
    "52-Week High": None,
    "52-Week Low": None,
    "Target Mean Price": None
}

if response_1.status_code == 200:
    stock_json_1 = response_1.json()

    if stock_json_1['quoteResponse']['result']:
        company_name = stock_json_1['quoteResponse']['result'][0]["longName"]
        current_price = stock_json_1['quoteResponse']['result'][0]["regularMarketPrice"]
        fifty_two_week_high = stock_json_1['quoteResponse']['result'][0]["fiftyTwoWeekHigh"]
        fifty_two_week_low = stock_json_1['quoteResponse']['result'][0]["fiftyTwoWeekLow"]
    else:
        print(f"Error: No data found for stock symbol '{stock}'.")
        company_name = current_price = fifty_two_week_high = fifty_two_week_low = None
else:
    print(f"Error: Unable to fetch data from {url_1} (Status code: {response_1.status_code})")
    company_name = current_price = fifty_two_week_high = fifty_two_week_low = None

url_2 = f"https://yfapi.net/v11/finance/quoteSummary/{stock}"
params_2 = {"modules": "financialData"}

response_2 = requests.get(url_2, headers=headers, params=params_2)

if response_2.status_code == 200:
    stock_json_2 = response_2.json()

    if stock_json_2['quoteSummary']['result']:
        try:
            target_mean_price = stock_json_2['quoteSummary']['result'][0]['financialData']['targetMeanPrice']['fmt']
        except KeyError:
            target_mean_price = "N/A"
    else:
        print(f"Error: No data found for financial data of stock symbol '{stock}'.")
        target_mean_price = "N/A"
else:
    print(f"Error: Unable to fetch data from {url_2} (Status code: {response_2.status_code})")
    target_mean_price = "N/A"

url_3 = f"https://yfapi.net/v1/finance/trending/US"
response_3 = requests.get(url_3, headers=headers)

if response_3.status_code == 200:
    trending_json = response_3.json()
    try:
        US_5_Trending = trending_json['finance']['result'][0]['quotes'][:5]
    except (KeyError, IndexError):
        US_5_Trending = []
else:
    print(f"Error: Unable to fetch data from {url_3} (Status code: {response_3.status_code})")
    US_5_Trending = []

if company_name and current_price is not None:
    print(f"{company_name} Price: ${current_price}, 52-Week High: ${fifty_two_week_high}, "
          f"52-Week Low: ${fifty_two_week_low}, Target Mean Price: {target_mean_price}")

if US_5_Trending:
    trending_stocks = ', '.join([stock['symbol'] for stock in US_5_Trending])
    print(f"Trending US Stocks: {trending_stocks}")
else:
    print("No trending stocks data available.")



Apple Inc. Price: $227.725, 52-Week High: $237.23, 52-Week Low: $164.08, Target Mean Price: 240.58
Trending US Stocks: AMSC, XRP-USD, RACE, PWR, BBD


In [45]:
Stock_df = pd.DataFrame([stock_data])
print(Stock_df)
for data in company_name, current_price, fifty_two_week_high, fifty_two_week_low, target_mean_price:
    print(data)

  Symbol Company Name Current Price 52-Week High 52-Week Low Target Mean Price
0   AAPL         None          None         None        None              None
Apple Inc.
227.725
237.23
164.08
240.58
